# import the data

$$
f=x^2 + \sqrt {y} + z
$$

In [730]:
import pandas as pd

In [731]:
df = pd.read_csv("data/result_testdata.csv", delimiter=",")
df

,x,y,z,f
0,0.876300,0.087160,0.678294,1.741426
1,0.506543,0.427771,0.117943,1.028571
2,0.469181,0.208947,0.862431,1.539668
3,0.997632,0.038999,0.628393,1.821145
4,0.680144,0.432733,0.571025,1.691445
...,...,...,...,...
569,0.213704,0.183989,0.577722,1.052330
570,0.727660,0.021602,0.820556,1.497021
571,0.452470,0.848223,0.896740,2.022459
572,0.470553,0.443268,0.204643,1.091847


# build the network

https://machinelearningmastery.com/develop-your-first-neural-network-with-pytorch-step-by-step/

In [732]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch import autograd
import math
from sklearn.metrics import mean_squared_error

In [733]:
X = df.iloc[:,:3]
y = df.iloc[:,3]
# y = df.iloc[:,:3].sum(axis=1) 

In [734]:
X = torch.tensor(X.values.astype(np.float32), requires_grad=True)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1)

In [735]:
X.shape

torch.Size([574, 3])

In [736]:
y.shape

torch.Size([574, 1])

In [737]:
class EstimateF(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden1 = nn.Linear(3, 160)
        self.act1 = nn.ReLU()
        self.hidden2 = nn.Linear(160, 80)
        self.act2 = nn.ReLU()
        self.output = nn.Linear(80, 16)
        self.act_output =  nn.Linear(16, 1)
 
    def forward(self, x):
        x = self.act1(self.hidden1(x))
        x = self.act2(self.hidden2(x))
        x = self.act_output(self.output(x))
        return x
 
model = EstimateF()
print(model)

EstimateF(
  (hidden1): Linear(in_features=3, out_features=160, bias=True)
  (act1): ReLU()
  (hidden2): Linear(in_features=160, out_features=80, bias=True)
  (act2): ReLU()
  (output): Linear(in_features=80, out_features=16, bias=True)
  (act_output): Linear(in_features=16, out_features=1, bias=True)
)


In [738]:
loss_fn = nn.MSELoss()  
optimizer = optim.Adam(model.parameters(), lr=0.001)

n_epochs = 500
batch_size = 10

for epoch in range(n_epochs):
    for i in range(0, len(X), batch_size):
        Xbatch = X[i:i+batch_size].clone()
        # Xbatch.requires_grad=True
        y_pred = model(Xbatch)
        ybatch = y[i:i+batch_size]
        f_x_t = autograd.grad(y_pred,Xbatch,torch.ones([Xbatch.shape[0], 1]), retain_graph=True, create_graph=True)[0] #first derivative
        loss = loss_fn(y_pred, ybatch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Finished epoch {epoch}, latest loss {loss}')

Finished epoch 0, latest loss 0.02473844774067402
Finished epoch 1, latest loss 0.0057207141071558
Finished epoch 2, latest loss 0.004246027674525976
Finished epoch 3, latest loss 0.003259702818468213
Finished epoch 4, latest loss 0.0018020868301391602
Finished epoch 5, latest loss 0.0007793891127221286
Finished epoch 6, latest loss 0.0006032128585502505
Finished epoch 7, latest loss 0.00047667903709225357
Finished epoch 8, latest loss 0.0003582267672754824
Finished epoch 9, latest loss 0.0002847350260708481
Finished epoch 10, latest loss 0.00025659165112301707
Finished epoch 11, latest loss 0.00021872295474167913
Finished epoch 12, latest loss 0.00019418285228312016
Finished epoch 13, latest loss 0.00016890194092411548
Finished epoch 14, latest loss 0.00015878220438025892
Finished epoch 15, latest loss 0.0001374365238007158
Finished epoch 16, latest loss 0.0001221296115545556
Finished epoch 17, latest loss 0.00011349030683049932
Finished epoch 18, latest loss 9.946194768417627e-05
Fin

In [739]:
# compute accuracy (no_grad is optional)
# with torch.no_grad():
#     y_pred = model(X)

X.requires_grad_()
X.retain_grad()
y_pred = model(X)

trainScore = math.sqrt(mean_squared_error(y_pred.detach().numpy(),y.detach().numpy()))
print('Train Score: %.2f RMSE' % (trainScore))

Train Score: 0.00 RMSE


# test the auto gradient

$$
f=x^2 + \sqrt {y} + z
$$

$$
f_x'=2x
$$

$$
f_y'=\frac{1}{2} * y ^ {-\frac{1}{2}}
$$

$$
f_z'=1
$$

In [740]:
X[0]

tensor([0.8763, 0.0872, 0.6783], grad_fn=<SelectBackward0>)

In [741]:
y[0]

tensor([1.7414])

In [742]:
model(X[0])

tensor([1.7396], grad_fn=<AddBackward0>)

In [743]:
autograd.grad(model(X),X,torch.ones([X.shape[0], 1]), retain_graph=True, create_graph=True)[0][0] #first derivative

tensor([1.6703, 1.6706, 1.0074], grad_fn=<SelectBackward0>)

good match!